In [7]:
import pandas as pd
import seaborn as sns
import pandasql as ps
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv("data/data.csv")

/Users/cheryljose/Documents/Projects/TestProject/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (37,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df = df[["X", "Y", "collision_date", "collision_time", "day_of_week", "intersection", "weather_1", "location_type", "collision_severity", "number_killed", "number_injured", "party_count", "primary_coll_factor", "pcf_viol_category", "hit_and_run", "type_of_collision", "road_surface", "road_cond_1", "lighting", "control_device", "pedestrian_accident", "bicycle_accident", "motorcycle_accident", "truck_accident", "alcohol_involved", "count_ped_killed", "count_ped_injured", "count_bicyclist_killed", "count_bicyclist_injured"]]

Fix missing Values

In [10]:
df['location_type']=df['location_type'].fillna('N')

df['pedestrian_accident']=df['pedestrian_accident'].fillna('N')

df['bicycle_accident']=df['bicycle_accident'].fillna('N')

df['motorcycle_accident']=df['motorcycle_accident'].fillna('N')

df['truck_accident']=df['truck_accident'].fillna('N')
df['alcohol_involved']=df['alcohol_involved'].fillna('N')

Drop Duplicates

In [11]:
rows_before = df.shape[0]

df = df.drop_duplicates()

"""rows before: {}, rows after: {}, duplicate rows: {}""". \
format(rows_before, df.shape[0], rows_before - df.shape[0])

'rows before: 171533, rows after: 171522, duplicate rows: 11'

Drop Rows without Geo Location

In [12]:
ps.sqldf("select ROUND(X, 0), ROUND(Y, 0), count(*) from df group by 1,2 order by 1 desc,2 desc", locals())

ps.sqldf("select * from df where ROUND(X, 0) = 0 or ROUND(Y, 0) = 0", locals())

rows_before = df.shape[0]
df = ps.sqldf("select * from df where ROUND(X, 0) != 0 and ROUND(Y, 0) != 0", locals())

"""rows before: {}, rows after: {}, rows dropped: {}""". \
format(rows_before, df.shape[0], rows_before - df.shape[0])

'rows before: 171522, rows after: 169515, rows dropped: 2007'

Get the Grid

In [36]:
df_1 = ps.sqldf("""
SELECT
1 AS sector,
-118.66 AS left,
-118.50 AS right,
34.32 AS up,
34.14 AS down
UNION ALL
SELECT
2 AS sector,
-118.50 AS left,
-118.34 AS right,
34.32 AS up,
34.14 AS down         
UNION ALL
SELECT
3 AS sector,
-118.36 AS left,
-118.14 AS right,
34.14 AS up,
34.05 AS down
UNION ALL
SELECT
4 AS sector,
-118.50 AS left,
-118.36 AS right,
34.14 AS up,
33.91 AS down 
UNION ALL
SELECT
5 AS sector,
-118.36 AS left,
-118.28 AS right,
34.05 AS up,
33.915 AS down
UNION ALL
SELECT
6 AS sector,
-118.28 AS left,
-118.13 AS right,
34.05 AS up,
33.915 AS down
UNION ALL
SELECT
7 AS sector,
-118.325 AS left,
-118.238 AS right,
33.915 AS up,
33.7 AS down

         """, locals())


location_lookup_df = ps.sqldf("""

SELECT
z2.*,
new_left + (new_right - new_left)/2 AS center_x,
new_down + (new_up - new_down)/2 AS center_y
FROM
    (SELECT
      sector,
      (delta_x*row1) + left AS new_left,
      (delta_x*(row1+1)) + left AS new_right,
      (delta_y*row2) + down AS new_down,
      (delta_y*(row2+1)) + down AS new_up,
      x.*,
      y.*,
      z.*

    FROM
        (SELECT
          sector,
          (right - left) / 10 delta_x,
          (up - down) / 10 delta_y,
          left,
          down,
          right,
          up
        FROM df_1
        ) x
    CROSS JOIN
        (
        SELECT 0 as row1 UNION ALL
        SELECT 1 as row1 UNION ALL
        SELECT 2 as row1 UNION ALL
        SELECT 3 as row1 UNION ALL
        SELECT 4 as row1 UNION ALL
        SELECT 5 as row1 UNION ALL
        SELECT 6 as row1 UNION ALL
        SELECT 7 as row1 UNION ALL
        SELECT 8 as row1 UNION ALL
        SELECT 9 as row1
        ) y 
        CROSS JOIN
        (
        SELECT 0 as row2 UNION ALL
        SELECT 1 as row2 UNION ALL
        SELECT 2 as row2 UNION ALL
        SELECT 3 as row2 UNION ALL
        SELECT 4 as row2 UNION ALL
        SELECT 5 as row2 UNION ALL
        SELECT 6 as row2 UNION ALL
        SELECT 7 as row2 UNION ALL
        SELECT 8 as row2 UNION ALL
        SELECT 9 as row2
        ) z
    ) z2
""", locals())
location_lookup_df['index1'] = location_lookup_df.index


Load datasets

In [37]:
dfA = pd.read_csv("data/2010_Census_Populations_by_Zip_Code.csv")
dfB = pd.read_csv("data/Intersections.csv")
dfC = pd.read_csv("data/Regulatory_Signs.csv")
dfD = pd.read_csv("data/Street_Lights.csv")
dfE = pd.read_csv("data/Stop_and_Yield_Signs.csv")
# dfF = pd.read_csv("data/Walkability_Index_Score_2012.csv")

In [43]:
df = ps.sqldf("""
select 
a.*, b.index1
from df a
INNER JOIN
location_lookup_df b
 ON a.X >= new_left AND a.X < new_right
AND a.Y >= new_down AND a.Y < new_up
""", locals())

In [44]:
from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()
# zipcode = search.by_zipcode(90001)
res = search.by_city(city="Los Angeles", returns=0)
search.export_to_csv(res, "result.csv")
dfAnew = pd.read_csv("data/result.csv")
dfA.columns = dfA.columns.str.strip().str.lower().str.replace(' ', '_')
dfAcombined = ps.sqldf("""
SELECT Population, Density, Wealthy, HouseOfUnits, Latitude, Longitude, median_age
    from dfAnew a
    LEFT JOIN
    dfA b ON
    a.Zipcode = b.zip_code
""", locals())
population_intermediate_df = ps.sqldf("""
select 
a.*,
b.*,
(((center_x - Longitude)*(center_x - Longitude)) + ((center_y - Latitude)*(center_y - Latitude))) AS distance
from location_lookup_df a
CROSS JOIN
dfAcombined b
""", locals())
population_final_df = ps.sqldf("""
select 
a.index1, Population, Density, Wealthy, HouseOfUnits, median_age
from population_intermediate_df a
INNER JOIN
(select index1, min(distance) AS min_distance from population_intermediate_df group by 1) b
ON a.index1 = b.index1 and
   a.distance = b.min_distance
group by 1
order by 1
""", locals())

In [45]:
intersections_df = ps.sqldf("""
select
index1, count(*) AS intersections
FROM
    (select index1, LAT, LON
    from dfB a
    INNER JOIN
    location_lookup_df b
     ON a.LON >= new_left AND a.LON < new_right
    AND a.LAT >= new_down AND a.LAT < new_up
    ) x
group by 1 
order by 1
""", locals())

street_lights_df = ps.sqldf("""
SELECT
  index1,
  SUM(total_watts) AS total_watts,
  SUM(total_lightbulbs) AS total_lightbulbs
  
FROM
(select index1, 
CASE WHEN TRIM(IFNULL(LAMPA,'')) = '' THEN 0 ELSE SUBSTR(LAMPA, 1, instr(LAMPA,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPB,'')) = '' THEN 0 ELSE SUBSTR(LAMPB, 1, instr(LAMPB,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPC,'')) = '' THEN 0 ELSE SUBSTR(LAMPC, 1, instr(LAMPC,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPD,'')) = '' THEN 0 ELSE SUBSTR(LAMPD, 1, instr(LAMPD,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPE,'')) = '' THEN 0 ELSE SUBSTR(LAMPE, 1, instr(LAMPE,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPF,'')) = '' THEN 0 ELSE SUBSTR(LAMPF, 1, instr(LAMPF,"W")) END AS total_watts,
CASE WHEN TRIM(IFNULL(LAMPA,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPB,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPC,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPD,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPE,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPF,'')) = '' THEN 0 ELSE 1 END AS total_lightbulbs
from dfD a
INNER JOIN
location_lookup_df b
 ON a.X >= new_left AND a.X < new_right
AND a.Y >= new_down AND a.Y < new_up
) x
GROUP BY 1
ORDER BY 1
""", locals())

regulatory_signs_df = ps.sqldf("""
SELECT
  index1,
  SUM(one_way_sign) AS one_way_signs,
  SUM(speed_limit_sign) AS speed_limit_signs,
  SUM(no_u_turn_sign) AS no_u_turn_signs
FROM
(select index1,
CASE WHEN lower(tooltip) like '%one way%' THEN 1 ELSE 0 END AS one_way_sign,
CASE WHEN lower(tooltip) like '%speed%' THEN 1 ELSE 0 END AS speed_limit_sign,
CASE WHEN lower(tooltip) like '%no u turn%' THEN 1 ELSE 0 END AS no_u_turn_sign
from dfC a
INNER JOIN
location_lookup_df b
 ON a.X >= new_left AND a.X < new_right
AND a.Y >= new_down AND a.Y < new_up
) x
GROUP BY 1
ORDER BY 1
""", locals())

stop_and_yield_signs_df = ps.sqldf("""
SELECT
  index1,
  SUM(stop_sign) AS stop_sign,
  SUM(yield_sign) AS yield_sign
FROM
(select index1,
    CASE WHEN lower(tooltip) like '%stop%' THEN 1 ELSE 0 END AS stop_sign,
    CASE WHEN lower(tooltip) like '%yield%' THEN 1 ELSE 0 END AS yield_sign
    from dfE a
    INNER JOIN
    location_lookup_df b
     ON a.X >= new_left AND a.X < new_right
    AND a.Y >= new_down AND a.Y < new_up
    ) x
GROUP BY 1
ORDER BY 1
""", locals())

In [47]:
final_granular_df = ps.sqldf("""
SELECT
*
FROM
df a
LEFT JOIN population_final_df b on b.index1 = a.index1
LEFT JOIN intersections_df c on c.index1 = a.index1
LEFT JOIN street_lights_df d on d.index1 = a.index1
LEFT JOIN regulatory_signs_df e on e.index1 = a.index1
LEFT JOIN stop_and_yield_signs_df f on f.index1 = a.index1
""", locals())

In [48]:
final_granular_df.shape

(166850, 48)

In [50]:
final_granular_df.to_csv("a.csv")